In [1]:
import pandas as pd
import json
from uuid import uuid4
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [2]:
with open("/Users/samreddyvarunreddy/Downloads/dryeye-token.json") as f:
    secrets = json.load(f)

In [3]:
CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

In [4]:
# === Secure Connect bundle path ===
cloud_config = {
    'secure_connect_bundle': '/Users/samreddyvarunreddy/Downloads/secure-connect-dryeye.zip'
}

In [5]:
# === Connect to Astra ===
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [6]:
# === Set keyspace ===
session.set_keyspace("dryeye")

In [ ]:
# === Create table (if it doesn't exist) ===
#session.execute("""
CREATE TABLE IF NOT EXISTS dry_eye_structured (
    id UUID PRIMARY KEY,
    gender TEXT,
    age INT,
    sleep_duration FLOAT,
    sleep_quality INT,
    stress_level INT,
    blood_pressure TEXT,
    heart_rate INT,
    daily_steps INT,
    physical_activity INT,
    height INT,
    weight INT,
    sleep_disorder TEXT,
    wake_up_during_night TEXT,
    feel_sleepy_during_day TEXT,
    caffeine_consumption TEXT,
    alcohol_consumption TEXT,
    smoking TEXT,
    medical_issue TEXT,
    ongoing_medication TEXT,
    smart_device_before_bed TEXT,
    average_screen_time FLOAT,
    blue_light_filter TEXT,
    discomfort_eye_strain TEXT,
    redness_in_eye TEXT,
    itchiness_irritation_in_eye TEXT,
    dry_eye_disease TEXT
);
""")

In [7]:
df = pd.read_csv("/Users/samreddyvarunreddy/Downloads/Dry_Eye_Dataset.csv")
df.columns = [col.strip().replace(" ", "_").replace("/", "_").replace("-", "_").lower() for col in df.columns]

In [8]:
df.head()

,gender,age,sleep_duration,sleep_quality,stress_level,blood_pressure,heart_rate,daily_steps,physical_activity,height,...,smoking,medical_issue,ongoing_medication,smart_device_before_bed,average_screen_time,blue_light_filter,discomfort_eye_strain,redness_in_eye,itchiness_irritation_in_eye,dry_eye_disease
0,F,24,9.5,2,1,137/89,67,3000,31,161,...,N,Y,Y,N,8.7,N,Y,Y,N,Y
1,M,39,9.6,2,3,108/64,60,12000,74,164,...,N,Y,Y,N,9.6,Y,N,N,Y,Y
2,F,45,5.4,1,5,134/81,95,12000,93,179,...,N,N,N,Y,4.0,N,Y,N,N,N
3,F,45,5.4,4,5,110/90,78,19000,32,160,...,N,Y,N,N,7.6,N,Y,N,Y,N
4,F,42,5.7,3,2,99/67,72,4000,173,179,...,N,Y,N,N,3.5,N,Y,Y,N,Y


In [9]:
df.describe()

,age,sleep_duration,sleep_quality,stress_level,heart_rate,daily_steps,physical_activity,height,weight,average_screen_time
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,31.422800,6.998245,2.997250,2.993750,79.912200,10536.900000,90.069750,174.865900,74.891850,5.519885
std,8.103717,1.731723,1.412283,1.407235,11.808279,5752.729186,52.317283,14.719903,14.733839,2.606305
min,18.000000,4.000000,1.000000,1.000000,60.000000,1000.000000,0.000000,150.000000,50.000000,1.000000
25%,24.000000,5.500000,2.000000,2.000000,70.000000,6000.000000,45.000000,162.000000,62.000000,3.300000
50%,31.000000,7.000000,3.000000,3.000000,80.000000,11000.000000,91.000000,175.000000,75.000000,5.500000
75%,39.000000,8.500000,4.000000,4.000000,90.000000,16000.000000,135.000000,188.000000,88.000000,7.800000
max,45.000000,10.000000,5.000000,5.000000,100.000000,20000.000000,180.000000,200.000000,100.000000,10.000000


In [10]:
print(df.isnull().sum())

gender                         0
age                            0
sleep_duration                 0
sleep_quality                  0
stress_level                   0
blood_pressure                 0
heart_rate                     0
daily_steps                    0
physical_activity              0
height                         0
weight                         0
sleep_disorder                 0
wake_up_during_night           0
feel_sleepy_during_day         0
caffeine_consumption           0
alcohol_consumption            0
smoking                        0
medical_issue                  0
ongoing_medication             0
smart_device_before_bed        0
average_screen_time            0
blue_light_filter              0
discomfort_eye_strain          0
redness_in_eye                 0
itchiness_irritation_in_eye    0
dry_eye_disease                0
dtype: int64


In [30]:
for _, row in df.iterrows():
    session.execute("""
        INSERT INTO dry_eye_csv (
            id, gender, age, sleep_duration, sleep_quality, stress_level,
            blood_pressure, heart_rate, daily_steps, physical_activity,
            height, weight, sleep_disorder, wake_up_during_night, feel_sleepy_during_day, caffeine_consumption, alcohol_consumption,
            smoking, medical_issue, ongoing_medication, smart_device_before_bed,
            average_screen_time, blue_light_filter, eye_strain,
            eye_redness, itchy_eye, dry_eye_disease
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                  %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        uuid4(), row.gender, row.age, row.sleep_duration, row.sleep_quality, row.stress_level,
        row.blood_pressure, row.heart_rate, row.daily_steps, row.physical_activity,
        row.height, row.weight, row.sleep_disorder, row.wake_up_during_night, row.feel_sleepy_during_day, row.caffeine_consumption,
        row.alcohol_consumption ,row.smoking, row.medical_issue, row.ongoing_medication, row.smart_device_before_bed,
        row.average_screen_time, row.blue_light_filter, row.discomfort_eye_strain,
        row.redness_in_eye , row.itchiness_irritation_in_eye, row.dry_eye_disease
    ))

In [11]:
# Create the subset table
session.execute("""
    CREATE TABLE IF NOT EXISTS dryeye_subset (
        id UUID PRIMARY KEY,
        age INT,
        gender TEXT,
        alcohol_consumption TEXT,
        smoking TEXT
    );
""")

In [12]:
# Select data from original table and insert into new one
rows = session.execute("SELECT id, age, gender, alcohol_consumption, smoking FROM dry_eye_csv")


In [13]:
for row in rows:
    session.execute("""
        INSERT INTO dryeye_subset (id, age, gender, alcohol_consumption, smoking)
        VALUES (%s, %s, %s, %s, %s)
    """, (row.id, row.age, row.gender, row.alcohol_consumption, row.smoking))

print(" Subset table 'dryeye_subset' created and populated.")

 Subset table 'dryeye_subset' created and populated.


THE CRUD OPERATIONS PERFORMED IN CQL CONSOLE, ARE ALSO EXECUTED IN PYTHON TO SHOW HOW THEY WORK. 

In [15]:
value = "Y"
rows = session.execute("""
    SELECT * FROM dry_eye_csv WHERE dry_eye_disease = %s LIMIT 10 ALLOW FILTERING
""", (value,))
print("🔍 Read result:")
for row in rows:
    print(dict(row._asdict()))


🔍 Read result:
{'id': UUID('6e58a856-7780-48a6-a467-42a25d82f307'), 'age': 23, 'alcohol_consumption': 'N', 'average_screen_time': 5.800000190734863, 'blood_pressure': '135/88', 'blue_light_filter': 'Y', 'caffeine_consumption': 'N', 'daily_steps': 16000, 'dry_eye_disease': 'Y', 'eye_redness': 'Y', 'eye_strain': 'N', 'feel_sleepy_during_day': 'N', 'gender': 'F', 'heart_rate': 63, 'height': 194.0, 'itchy_eye': 'N', 'medical_issue': 'N', 'ongoing_medication': 'Y', 'physical_activity': 158, 'sleep_disorder': 'N', 'sleep_duration': 7.0, 'sleep_quality': 1, 'smart_device_before_bed': 'N', 'smoking': 'N', 'stress_level': 3, 'wake_up_during_night': 'N', 'weight': 73.0}
{'id': UUID('129d8feb-eda3-4560-93d5-17c903fde60f'), 'age': 41, 'alcohol_consumption': 'Y', 'average_screen_time': 6.300000190734863, 'blood_pressure': '136/83', 'blue_light_filter': 'Y', 'caffeine_consumption': 'Y', 'daily_steps': 7000, 'dry_eye_disease': 'Y', 'eye_redness': 'N', 'eye_strain': 'Y', 'feel_sleepy_during_day': 'Y',

In [19]:
# CREATE (Insert a new row)
new_id = uuid4()
session.execute("""
    INSERT INTO dry_eye_csv (
        id, age, gender, sleep_duration, sleep_quality, stress_level,
        blood_pressure, heart_rate, daily_steps, physical_activity,
        height, weight, sleep_disorder, wake_up_during_night, feel_sleepy_during_day,
        caffeine_consumption, alcohol_consumption, smoking, medical_issue,
        ongoing_medication, smart_device_before_bed, average_screen_time,
        blue_light_filter, eye_strain, eye_redness,
        itchy_eye, dry_eye_disease
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", (
    new_id, 35, 'F', 7.0, 4, 2, 'Normal', 75, 5000, 3,
    160.0, 55.0, 'None', 'No', 'No', 'Low', 'None', 'N', 'None',
    'No', 'Yes', 6.5, 'Yes', 'Low', 'None', 'No', 'Y'
))
print(f" Inserted new row with ID: {new_id}")

 Inserted new row with ID: da6a087a-f96f-4bd7-9f37-30922452a8cd


In [20]:
row = session.execute("SELECT * FROM dry_eye_csv WHERE id = %s", (new_id,)).one()
if row:
    print(" Retrieved:", dict(row._asdict()))
else:
    print(" No row found.")


 Retrieved: {'id': UUID('da6a087a-f96f-4bd7-9f37-30922452a8cd'), 'age': 35, 'alcohol_consumption': 'None', 'average_screen_time': 6.5, 'blood_pressure': 'Normal', 'blue_light_filter': 'Yes', 'caffeine_consumption': 'Low', 'daily_steps': 5000, 'dry_eye_disease': 'Y', 'eye_redness': 'None', 'eye_strain': 'Low', 'feel_sleepy_during_day': 'No', 'gender': 'F', 'heart_rate': 75, 'height': 160.0, 'itchy_eye': 'No', 'medical_issue': 'None', 'ongoing_medication': 'No', 'physical_activity': 3, 'sleep_disorder': 'None', 'sleep_duration': 7.0, 'sleep_quality': 4, 'smart_device_before_bed': 'Yes', 'smoking': 'N', 'stress_level': 2, 'wake_up_during_night': 'No', 'weight': 55.0}


In [21]:
session.execute("""
    UPDATE dry_eye_csv SET stress_level = %s, sleep_quality = %s WHERE id = %s
""", (5, 3, new_id))
print(" Updated stress_level and sleep_quality")

 Updated stress_level and sleep_quality


In [22]:
session.execute("DELETE FROM dry_eye_csv WHERE id = %s", (new_id,))
print("Deleted the row.")

Deleted the row.


In [23]:
rows = session.execute("SELECT * FROM dry_eye_csv LIMIT 5")
for row in rows:
    print(dict(row._asdict()))

{'id': UUID('6e58a856-7780-48a6-a467-42a25d82f307'), 'age': 23, 'alcohol_consumption': 'N', 'average_screen_time': 5.800000190734863, 'blood_pressure': '135/88', 'blue_light_filter': 'Y', 'caffeine_consumption': 'N', 'daily_steps': 16000, 'dry_eye_disease': 'Y', 'eye_redness': 'Y', 'eye_strain': 'N', 'feel_sleepy_during_day': 'N', 'gender': 'F', 'heart_rate': 63, 'height': 194.0, 'itchy_eye': 'N', 'medical_issue': 'N', 'ongoing_medication': 'Y', 'physical_activity': 158, 'sleep_disorder': 'N', 'sleep_duration': 7.0, 'sleep_quality': 1, 'smart_device_before_bed': 'N', 'smoking': 'N', 'stress_level': 3, 'wake_up_during_night': 'N', 'weight': 73.0}
{'id': UUID('129d8feb-eda3-4560-93d5-17c903fde60f'), 'age': 41, 'alcohol_consumption': 'Y', 'average_screen_time': 6.300000190734863, 'blood_pressure': '136/83', 'blue_light_filter': 'Y', 'caffeine_consumption': 'Y', 'daily_steps': 7000, 'dry_eye_disease': 'Y', 'eye_redness': 'N', 'eye_strain': 'Y', 'feel_sleepy_during_day': 'Y', 'gender': 'M',

In [24]:
result = session.execute("SELECT COUNT(*) FROM dryeye.dry_eye_csv")
count = result.one()[0]
print(f" Total rows: {count}")


 Total rows: 20001
